# Cleaning script

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lucasulsamer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Two datasets of tweets gathered at two timepoints are combined and duplicates are droppen.

In [2]:
dat1 = pd.read_csv('tweet_test.csv').iloc[: , 1:]
dat2 = pd.read_csv('tweets2.csv').iloc[: , 1:]

dat_complete = dat1.append(dat2, ignore_index=True).drop_duplicates()
dat_complete.columns = ['tweet_id', 'username', 'text', 'tweet_source', 'tweet_source_link', 'rt_count', 'fav_count']
dat_complete.drop_duplicates(subset='text',inplace=True)

,0,1,2,3,4,5,6
0,1468565079838842880,bosch_stephan,@Buitenbeen1 Linkse klootzakken die almaar lop...,Twitter for iPhone,http://twitter.com/download/iphone,0,0
1,1468564565092876297,Tweelingetje2V,"RT @RvRavenstijn: Ja, krankzinnig, maar dat he...",Twitter Web App,https://mobile.twitter.com,1,0
2,1468564477197135876,DeArcheoloog,@paulcaballo2 @k_steen01 KOZP heeft niets met ...,Twitter Web App,https://mobile.twitter.com,0,0
3,1468563864312766469,pim0605a,RT @moeva18: Nu er zo ontzettend veel asielzoe...,Twitter Web App,https://mobile.twitter.com,15,0
4,1468563333641084931,jehmsstudio,"RT @anne_kervers: ""KOZP heeft door de jaren he...",Twitter Web App,https://mobile.twitter.com,9,0
...,...,...,...,...,...,...,...
26071,1465372897757868036,HumbleNotHubris,RT @BilalDelpesche: @malukutjj @tuigvanzeventi...,Twitter for Android,http://twitter.com/download/android,2,0
26072,1465372530504519680,WiesCools,RT @pvvstadutrecht: Sinterklaasfeest bij de Vo...,Twitter for Android,http://twitter.com/download/android,103,0
26073,1465370624709251080,Ron62237731,@JazzGotBlues @Saartje63781665 @lindavoortman ...,Twitter for Android,http://twitter.com/download/android,0,4
26074,1465368893216677889,charsnuff,Helden van de voedselbank weten hoe het hoort\...,Instagram,http://instagram.com,0,2


## Create cleaned dataset for rt-only dataset
Includes the creation of an adjacency list for all retweets

In [12]:
# RT ONLY
dat_complete_rtonly = dat_complete[dat_complete.text.str.contains("^RT")]

dat_complete_rtonly = dat_complete_rtonly.reset_index()

# Create cleaned text
additional  = ['rt','rts','retweet', 'RT']
swords = set().union(stopwords.words('dutch'),additional)

dat_complete_rtonly['clean_text'] = dat_complete_rtonly['text'].str.lower()\
          .str.replace('(@[a-z0-9]+)\w+',' ')\
          .str.replace('(http\S+)', ' ')\
          .str.replace('([^0-9a-z \t])',' ')\
          .str.replace(' +',' ')\
          .apply(lambda x: [i for i in x.split() if not i in swords])

dat_complete_rtonly.to_csv('cleaned_rtonly.csv', index=False)

# Create mentions
dat_complete_rtonly['mentions'] = ""

# Create empty list
output = []

for index, row in dat_complete_rtonly.iterrows():       
    if len(re.findall("@([a-zA-Z0-9_]{1,50})", dat_complete_rtonly.loc[index]['text'])) == 0:
        output.append(row.tolist())
    else:
        sub = row
        sub['mentions'] = [re.findall("@([a-zA-Z0-9_]{1,50})", sub['text'])[0]]
        output.append(sub.tolist())
        
columns = ['tweet_id', 'username', 'text', 'tweet_source', 'tweet_source_link', 'rt_count', 'fav_count', 'clean_text', 'mentions']
dat_complete_rtonly = pd.DataFrame(output).iloc[:,1:]; dat_complete_rtonly.columns = columns    

pd.DataFrame(adjacency_list, columns=['source', 'target']).query("source != target").to_csv('cleaned_rtonly_adj.csv', index=False)

/var/folders/3q/gtdr3t3530ggtztvm2wvp1mc0000gn/T/ipykernel_7627/3459962863.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  dat_complete_rtonly['clean_text'] = dat_complete_rtonly['text'].str.lower()\


## Create cleaned dataset for no rt dataset
Includes the creation of an adjacency list for all mentions excluding retweets

In [7]:
dat_complete_nort = dat_complete[~dat_complete.text.str.contains("^RT")]; dat_complete_nort

# Create cleaned text
additional  = ['rt','rts','retweet']
swords = set().union(stopwords.words('dutch'),additional)

dat_complete_nort['clean_text'] = dat_complete_nort['text'].str.lower()\
          .str.replace('(@[a-z0-9]+)\w+',' ')\
          .str.replace('(http\S+)', ' ')\
          .str.replace('([^0-9a-z \t])',' ')\
          .str.replace(' +',' ')\
          .apply(lambda x: [i for i in x.split() if not i in swords])

dat_complete_nort.to_csv('cleaned_no_rt.csv', index=False); dat_complete_nort

# Create mentions
dat_complete_nort['mentions'] = dat_complete_nort['text'].str.findall("@([a-zA-Z0-9_]{1,50})")

adjacency_list = []

for index, row in dat_complete_nort.iterrows():
    source = row['username']
    
    adjacency = []
    
    for mention in range(0, len(row['mentions'])):
        adjacency.append([source, row['mentions'][mention]])
        
    if len(adjacency) > 0:
        for i in range(len(adjacency)):
            adjacency_list.append(adjacency[i])
    else:
        continue
        
pd.DataFrame(adjacency_list, columns=['source', 'target']).query("source != target").to_csv('cleaned_no_rt_adj.csv', index=False)

/var/folders/3q/gtdr3t3530ggtztvm2wvp1mc0000gn/T/ipykernel_7627/2669809704.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  dat_complete_nort['clean_text'] = dat_complete_nort['text'].str.lower()\
/var/folders/3q/gtdr3t3530ggtztvm2wvp1mc0000gn/T/ipykernel_7627/2669809704.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat_complete_nort['clean_text'] = dat_complete_nort['text'].str.lower()\
/var/folders/3q/gtdr3t3530ggtztvm2wvp1mc0000gn/T/ipykernel_7627/2669809704.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

Source: Dutchfan33
Mention: SylvanaBIJ1
Source: blattae2
Source: robknijff27
Mention: yungquichot
Mention: BartNijman
Source: bart_sluis
Source: Jaaph
Mention: jesseklaver
Source: Mapacho111
Mention: jesseklaver
Source: NederbelgJ
Mention: SUGenocide
Mention: oornstein
Mention: kozwartepiet
Source: JanKeyzer2
Mention: SylvanaBIJ1
Source: CoKatana
Source: GraafR2
Mention: jesseklaver
Source: Pattysaurus_
Mention: jesseklaver
Source: elmer079
Mention: robhonig
Source: FiliaVeritas
Mention: W0mmetje
Mention: HartvNL
Source: goort1409
Source: PatWeening
Mention: jasonwalters85
Mention: TheRebelThePoet
Source: MikeEhrmantrau9
Mention: Judge79564714
Mention: NadiaBouras
Mention: soulsister4u
Source: NederbelgJ
Mention: SUGenocide
Mention: oornstein
Mention: kozwartepiet
Source: EinsteinBarbie
Source: bentualdepineut
Source: Schilinde
Mention: DenKozp
Mention: SylvanaBIJ1
Source: Dewik2013
Mention: jesseklaver
Source: Alphae1978
Mention: DeMariekeK
Mention: cirstenkot
Mention: LiekeSievers
Me

Mention: NHNieuws
Source: DincelBerna
Source: ZwaanBente
Mention: Franssen1956
Mention: kimvankeken
Mention: KeesVlaar
Mention: JanRiezenkamp
Source: CovidNewsNL
Source: Adijksma1985
Mention: GWK112
Mention: kozwartepiet
Source: hugo_lasnier
Mention: DeAltijd
Mention: Het_OM
Mention: POL_Volendam
Source: MvButselaar
Mention: iAnnetonly
Source: snldrs
Mention: elineannepeters
Source: jannavandam
Mention: FeyeNody
Mention: TheRebelThePoet
Source: PeterPos1
Mention: OTTO45520061
Mention: heinsjohn
Mention: lotterik2
Mention: frankvdlinde
Source: Ahimsa147
Source: FrankDijkstra4
Mention: Mirjam152
Source: 020bowonor
Source: mayaVirg
Mention: HuubBellemakers
Mention: M_Meijer
Source: fireondemand1
Source: DirkSellsAHouse
Mention: SmeetsLaw
Mention: GJHAmsterdam
Mention: Het_OM
Source: Tobinator4991
Mention: DeStefVolendam
Source: amal_chatt
Source: Christovte1
Mention: SylvanaBIJ1
Source: ObmaPascal
Mention: MvButselaar
Mention: Schnoetsz
Source: BartNijman
Source: DrDre_010
Mention: JohnSt

Source: sas_pvv_
Source: vgwhz
Mention: SylvanaBIJ1
Source: JdeWittvanOldeB
Mention: SylvanaBIJ1
Source: TommyGligoor
Source: jetje76
Mention: SteenksRafaela
Source: TheRealDonInigo
Source: ziggy0001
Mention: TjerkLangman
Source: NaughtiusNL
Mention: BaukeButs
Mention: Dewereldopzijn1
Source: kkrutte
Source: colourbird00
Source: qgmeghan
Mention: chartdata
Mention: BrunoMars
Source: sjoeba666
Mention: TofikDibi
Mention: jaces
Source: mraymon34794331
Mention: erikpjverweij
Source: JamesRon1980
Mention: teigemans
Mention: DonJohannes1
Mention: verfben
Mention: NHNieuws
Mention: DenKozp
Mention: TheRebelThePoet
Mention: PolitiekBIJ1
Mention: AmsterdamBIJ1
Source: MarlonReina
Source: John2B77
Mention: UbedaMichel
Source: assassinXXXL
Mention: SylvanaBIJ1
Source: oornstein
Mention: wierdduk
Source: ArchieRM20
Mention: erikpjverweij
Mention: eetschrijver
Mention: op1npo
Mention: jortkelder
Source: mraymon34794331
Mention: LavieJanRoos
Source: TheYazzThing
Source: JamesRon1980
Mention: DonJoh

Source: NuRevolte
Source: Chestman225
Mention: ShashiRoopram
Mention: THBO1970
Source: WimWonders
Mention: EelcoHoecke
Source: KoopsJoey
Source: jorispinter3
Mention: fuckroeper
Source: Lewis_GS_
Source: antivirusje
Mention: NHNieuws
Source: JDOS_ICT_SOL
Mention: KarelDoorman3
Source: Overmars12
Mention: alexafca
Mention: LavieJanRoos
Source: hoppakee13
Mention: thyztwizta
Mention: MwTuinstra
Source: JamesRon1980
Mention: sander_tweets
Source: Koos24Koos
Mention: TimVergoossen
Mention: frankvdlinde
Source: Mark1908Fr
Source: Roos61302990
Source: Koos24Koos
Mention: Martijnmbrmrbr
Mention: Mevrouw_Maas
Mention: mjppeeters
Mention: NHNieuws
Source: ferdyroet
Mention: FeyeNody
Mention: TheRebelThePoet
Source: zeeschilder
Mention: Mirjam152
Source: Hofscana
Mention: sander_tweets
Source: michel50
Source: patriciakooreva
Mention: RaafMeneer
Mention: NOS
Mention: NOS
Source: NicVis4
Mention: BPeppelman
Mention: gronkk
Mention: erikpjverweij
Source: BasB053
Mention: Matthijstijd
Mention: jade

...

Mention: kooswijnands
Mention: telegraaf
Source: CommuniterHold1
Mention: Donkerwoud
Mention: EdwardVeerman
Source: Loupelamp
Mention: Jkr653
Mention: DeSloperT
Source: HvanderVen_
Source: DunK_058
Mention: robinkaljouw
Mention: BartNijman
Source: GirbeCameron
Source: BrianMa84334383
Source: Deon10000
Mention: Levensgeniete19
Source: _Salticidae_
Source: dalli66
Mention: HartvNL
Source: NeilMachia
Mention: LindaSchD
Source: TheDutch_Dude83
Source: Cover2412
Source: jessiejessie007
Source: ctr_alt_forward
Mention: petervlemmix
Mention: OmroepON
Source: Zilverslang1
Source: LeuterDe
Mention: HartvNL
Source: dezusvandeboer
Mention: vriesbgde
Source: Koos24Koos
Mention: RutgerGW
Source: gewoondoorgaan2
Mention: DongaMaurits
Mention: GozerOost
Source: gwdubbeldam
Source: PelleBerting
Mention: Teletekst
Source: DeleteUrQRcode
Source: FenixWatson
Source: DMWorldLAev
Source: leroy_kakiay
Source: Wellofgrief
Source: Gerben98551608
Mention: Boldking1973
Source: BrianMa84334383
Source: Rent_an_Id

Source: Marlon_93_D
Mention: GelderToi
Mention: op1npo
Source: martin_joes
Source: based_tr
Mention: EdWres
Source: GelderToi
Mention: op1npo
Source: hollandswiefke
Mention: NPO1
Source: Cornelia1949
Mention: Lidnr6
Mention: LauraBromet
Mention: LauraBromet
Source: Jetze79
Mention: op1npo
Source: ZwartePiet_be
Mention: booth_brian
Source: de_rekenmeester
Mention: ZihniOzdil
Mention: baspaternotte
Source: R_de_Collier
Mention: _pipke_
Source: tisaltijdwa
Mention: gjavanputten
Mention: VermeulenHellen
Mention: ADnl
Source: Marcje2013
Mention: TheRebelThePoet
Source: JoinMyTeamBB
Mention: cispeters35
Mention: isgoedhoor
Mention: GroningerHopper
Mention: op1npo
Mention: Nadine_Ridder
Mention: amsterdamlale
Mention: nspublieksprijs
Mention: RenkemaErica
Source: EllieB87980022
Mention: Telegraaf
Source: correctietikje
Mention: erikstronks
Source: nollie123
Mention: LauraBromet
Source: Lilylilajo
Mention: robertpijkeren
Source: nulambda7
Mention: AHMEDELBAHRI
Mention: MauvaisCoton
Mention: Zw

Mention: TheRebelThePoet
Source: romije72
Mention: jesseklaver
Source: Niem62453560Ano
Mention: THiddema
Mention: TheRebelThePoet
Source: hardcorejunkyxx
Mention: Bart21329430
Source: ArmyFvd
Source: j_tijman
Mention: Morpheusz5
Mention: ZilteBotte
Source: JoWe_FBPEGlobal
Mention: JohnMullem
Mention: Klimaatactie
Mention: volkskrant
Mention: kozwartepiet
Mention: POL_Volendam
Mention: POL_Volendam
Source: juvadewe
Mention: trottlesnot
Mention: KDaver7
Mention: LaurensJan4
Mention: Leolucassen
Source: SapperDeKriek
Mention: jesseklaver
Source: JoWe_FBPEGlobal
Mention: JohnMullem
Mention: Klimaatactie
Mention: volkskrant
Mention: kozwartepiet
Mention: POL_Volendam
Mention: kozwartepiet
Source: peter_schijven
Mention: richardstwitz
Mention: SergeTwit2
Mention: Zwarte_Magica
Source: jaekuik1
Mention: kozwartepiet
Mention: Heerbean
Source: juvadewe
Mention: trottlesnot
Mention: KDaver7
Mention: LaurensJan4
Mention: Leolucassen
Source: JoWe_FBPEGlobal
Mention: JohnMullem
Mention: Klimaatacti

Mention: vlbelang
Mention: bol_com
Mention: Google
Mention: Filip_Bru
Source: verbeekjca
Mention: Hiker4001
Source: salvatore1976cs
Source: HJEbbelink
Mention: AnnemarieGelder
Source: donutj54
Mention: HartvNL
Source: ghf1948
Source: baldsarcasm
Source: IckxsI
Mention: kozwartepiet
Mention: TheRebelThePoet
Source: IckxsI
Mention: kozwartepiet
Mention: TheRebelThePoet
Source: Benjamin___Ben
Mention: erikpjverweij
Source: F_r_e_n_k_i_e
Mention: Orangeazahar
Mention: shvdveen
Source: harryfmik
Mention: henkbres
Mention: Volger66082047
Source: Boris31209746
Mention: mariabauwens
Mention: ShashiRoopram
Source: Hvbt36Henny
Mention: B_renda
Source: valk90
Mention: ADnl
Mention: Freddy19831306
Source: marcelzegtniets
Mention: Bariant79
Source: KaironEdwards
Source: DingesZegt
Mention: HartvNL
Source: IckxsI
Mention: kozwartepiet
Mention: TheRebelThePoet
Source: MolenaarAnnet
Mention: ShashiRoopram
Source: ReitsLydia
Source: ZatharoZ
Mention: fabel31577
Source: MolenaarAnnet
Mention: jesseklave

Mention: Feetje62
Mention: SandroP71216998
Mention: moeva18
Source: oldnoskool
Source: ceesiper
Source: FugelVon
Mention: FvdMosselaar
Source: SanderGrootDe
Source: JanBos38966583
Source: PeterBuscher
Mention: HartvNL
Source: NickBlunt10
Source: CaliMer65307482
Source: UitIJsselmonde
Source: DeGelderlander
Source: Lyn29090700
Mention: PeterBeuker
Mention: jesseklaver
Source: iezjeiez
Mention: PatriciaFoort
Source: Dolfynco
Mention: TheRebelThePoet
Source: jankuhn81
Mention: jesseklaver
Source: Gaby61407817
Mention: TPOnl
Mention: Hannesz1956
Source: Jerky16976705
Source: DickKraaij
Mention: WhiteangelBe
Mention: HartvNL
Source: NorahJtheCat
Mention: TailfoxTim
Source: GeesjevanD
Mention: rod_jewel
Mention: erikpjverweij
Source: EricMirko_
Mention: GeraldVieze
Mention: GijsBaller
Mention: FeyeNody
Mention: TheRebelThePoet
Source: Hngstmngl
Mention: LichtDerRede
Mention: EinsteinBarbie
Mention: based_tr
Mention: AquariusInRed
Source: EMRoofer
Source: Gaby61407817
Mention: HartvNL
Source:

## Create cleaned dataset for the complete dataset
Includes the creation of an adjacency list for all meentions and retweets

In [18]:
dat_complete_rt = dat_complete.reset_index()

# Create cleaned text
additional  = ['rt','rts','retweet']
swords = set().union(stopwords.words('dutch'),additional)

dat_complete_rt['clean_text'] = dat_complete_rt['text'].str.lower()\
          .str.replace('(@[a-z0-9]+)\w+',' ')\
          .str.replace('(http\S+)', ' ')\
          .str.replace('([^0-9a-z \t])',' ')\
          .str.replace(' +',' ')\
          .apply(lambda x: [i for i in x.split() if not i in swords])

dat_complete_rt.to_csv('cleaned_rt.csv', index=False)

# Create mentions
dat_complete_rt['mentions'] = ""

# Create empty list
output = []

for index, row in dat_complete_rt.iterrows():       
    if 'RT ' in row['text'][:3]:
        if len(re.findall("@([a-zA-Z0-9_]{1,50})", dat_complete_rt.loc[index]['text'])) == 0:
            output.append(row.tolist())
        else:
            sub = row
            sub['mentions'] = [re.findall("@([a-zA-Z0-9_]{1,50})", sub['text'])[0]]
            output.append(sub.tolist())
            
    else:
        sub = row
        sub['mentions'] = re.findall("@([a-zA-Z0-9_]{1,50})", sub['text'])
        output.append(sub.tolist())
        

columns = ['tweet_id', 'username', 'text', 'tweet_source', 'tweet_source_link', 'rt_count', 'fav_count', 'clean_text', 'mentions']
dat_complete_rt = pd.DataFrame(output).iloc[:,1:]; dat_complete_rt.columns = columns

adjacency_list = []

for index, row in dat_complete_rt.iterrows():
    source = row['username']

    adjacency = []

    for mention in range(0, len(row['mentions'])):
        adjacency.append([source, row['mentions'][mention]])

    if len(adjacency) > 0:
        for i in range(len(adjacency)):
            adjacency_list.append(adjacency[i])
    else:
        continue

pd.DataFrame(adjacency_list, columns=['source', 'target']).query("source != target").to_csv('cleaned_rt_adj.csv', index=False

/var/folders/3q/gtdr3t3530ggtztvm2wvp1mc0000gn/T/ipykernel_3053/42267448.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  dat_complete_rt['clean_text'] = dat_complete_rt['text'].str.lower()\
